In [31]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [33]:
# Use absolute path or verify the file exists
green_taxi = pd.read_parquet('green_tripdata_2025-11.parquet')
zones = pd.read_csv('taxi_zone_lookup.csv')

In [34]:
green_taxi.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [ ]:
green_taxi.dtypes

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'cbd_congestion_fee'],
      dtype='object')

In [55]:
# SELECT COUNT(*) FROM green_taxi WHERE trip_distance <= 1

filtered = green_taxi[(green_taxi['trip_distance'] <= 1) & 
					  (green_taxi["lpep_pickup_datetime"] >= "2025-11-01") & 
					  (green_taxi["lpep_pickup_datetime"] < "2025-12-01")]
count = len(filtered)
print(count)

8007


In [59]:
#Q4 Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors). 
#   Use the pick up time for your calculations.

# df of trips with trip_distance less than 100 miles
valid = green_taxi[green_taxi["trip_distance"] < 100]

# longest trip distance
idx = valid["trip_distance"].idxmax()

# pick up day with the longest trip distance
pickup_day = valid.loc[idx, "lpep_pickup_datetime"].date()
longest_distance = valid.loc[idx, "trip_distance"]

pickup_day, longest_distance

(datetime.date(2025, 11, 14), np.float64(88.03))

In [61]:
#Q4 Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors). 
#   Use the pick up time for your calculations.

green_taxi["pickup_day"] = green_taxi["lpep_pickup_datetime"].dt.date

valid = green_taxi[green_taxi["trip_distance"] < 100]

longest_per_day = (
    valid.groupby("pickup_day")["trip_distance"]
         .max()
         .sort_index()
)

day_of_longest_trip = longest_per_day.idxmax()
distance_of_longest_trip = longest_per_day.max()

day_of_longest_trip, distance_of_longest_trip

(datetime.date(2025, 11, 14), np.float64(88.03))

In [72]:
zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [68]:
# Q5:Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?

# 1. Filter to Nov 18, 2025
day = green_taxi[
    (green_taxi["lpep_pickup_datetime"] >= "2025-11-18") &
    (green_taxi["lpep_pickup_datetime"] <  "2025-11-19")
]

# 2. Join with zones lookup
df = day.merge(
    zones,
    left_on="PULocationID",
    right_on="LocationID",
    how="left"
)

# 3. Sum total_amount by pickup zone
zone_totals = (
    df.groupby("Zone")["total_amount"]
      .sum()
      .sort_values(ascending=False)
)

# 4. Pickup zone with the largest total_amount
zone_totals.head(1)

Zone
East Harlem North    9281.92
Name: total_amount, dtype: float64

In [73]:
# For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

# Note: it's tip , not trip. We need the name of the zone, not the ID.

# 1. Filter to November 2025
nov = green_taxi[
    (green_taxi["lpep_pickup_datetime"] >= "2025-11-01") &
    (green_taxi["lpep_pickup_datetime"] <  "2025-12-01")
]

# 2. Join to get pickup zone names
df = nov.merge(zones, left_on="PULocationID", right_on="LocationID", how="left")

# 3. Filter to East Harlem North pickups
df = df[df["Zone"] == "East Harlem North"]

# 4. Join again to get dropoff zone names
df = df.merge(
    zones.add_prefix("do_"),
    left_on="DOLocationID",
    right_on="do_LocationID",
    how="left"
)

# 5. Dropoff zone with largest total tip
df.groupby("do_Zone")["tip_amount"].sum().sort_values(ascending=False).head(1)

do_Zone
Upper East Side North    4242.01
Name: tip_amount, dtype: float64